## 1) Import required libraries - 2 Mark

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
from nltk.corpus import stopwords
import nltk 
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import math
#from googletrans import Translator
nltk.download('words')
from nltk.corpus import words 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
#from sklearn.cluster import MiniBatchKMeans
#!pip install -q wordcloud
#!pip install -q kmeans
import wordcloud


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\pjatthi\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
# read the facebook dataset 
df = pd.read_csv('Corona_NLP_train.csv',encoding='latin1',parse_dates=['TweetAt'])

In [3]:
df.shape

(41157, 6)

In [4]:
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,2020-03-16,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,2020-03-16,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,2020-03-16,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,2020-03-16,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,2020-03-16,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [5]:
df.dtypes

UserName                  int64
ScreenName                int64
Location                 object
TweetAt          datetime64[ns]
OriginalTweet            object
Sentiment                object
dtype: object

In [6]:
df.describe(include=['object','datetime'])

<ipython-input-6-2dd22356947c>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe(include=['object','datetime'])


,Location,TweetAt,OriginalTweet,Sentiment
count,32567,41157,41157,41157
unique,12220,30,41157,5
top,London,2020-03-20 00:00:00,A byproduct of COVID-19: #cyberfraud. Now is t...,Positive
freq,540,3448,1,11422
first,NaN,2020-01-04 00:00:00,NaN,NaN
last,NaN,2020-12-04 00:00:00,NaN,NaN


In [7]:
df_subset = df.drop(['UserName','ScreenName','TweetAt','Location'], axis=1)

In [8]:
df_subset.head(5)

,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,advice Talk to your neighbours family to excha...,Positive
2,Coronavirus Australia: Woolworths to give elde...,Positive
3,My food stock is not the only one which is emp...,Positive
4,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [9]:
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   OriginalTweet  41157 non-null  object
 1   Sentiment      41157 non-null  object
dtypes: object(2)
memory usage: 643.2+ KB


In [10]:
df_subset.isnull().sum()

OriginalTweet    0
Sentiment        0
dtype: int64

## 2) Read dataset and perfom Text processing for the tweets ( Remove Stop words , special characters and convert the text to lowercase ) - 3 Mark

In [11]:
wordnet_lemmatizer = WordNetLemmatizer()
word = list(set(words.words()))
from nltk.stem import PorterStemmer
porter = PorterStemmer()
def textCleaning(text):
    text = text.lower()
    #text = " ".join([w for w in nltk.wordpunct_tokenize(text) if w.lower() in words or not w.isalpha()])
    #text = word_tokenize(text)
    ## use regular expression to remove url from the data
    text = re.sub(r"http\S+", "", text)
    ## remove mentions, hashtag and emojios from the data and also remove the non english characters
    text = ' '.join(re.sub("(@[A-Za-z0–9]+)|([^-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())
    text = " ".join([porter.stem(i) for i in word_tokenize(text)])
    ## remove punctuation from the data
    text = " ".join([i for i in word_tokenize(text) if i not in string.punctuation])
    ## remove numbers 
    text = " ".join([i for i in word_tokenize(text) if not i.isdigit()])
    ## remove stopwords from the data
    text = " ".join([i for i in word_tokenize(text) if i not in set(stopwords.words('english'))])
    text = " ".join([wordnet_lemmatizer.lemmatize(i) for i in word_tokenize(text)])
   
    return text

In [12]:
df_subset['clean_tweets'] = df_subset.OriginalTweet.apply(lambda x: textCleaning(x))

In [13]:
df_subset.to_csv('cleanData.csv')

In [14]:
temp = df_subset.clean_tweets[~(df_subset.clean_tweets == '')]

In [15]:
import pandas as pd
import re
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import nltk
from sklearn.naive_bayes import GaussianNB

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV

In [16]:
print(df_subset)

                                           OriginalTweet           Sentiment  \
0      @MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...             Neutral   
1      advice Talk to your neighbours family to excha...            Positive   
2      Coronavirus Australia: Woolworths to give elde...            Positive   
3      My food stock is not the only one which is emp...            Positive   
4      Me, ready to go at supermarket during the #COV...  Extremely Negative   
...                                                  ...                 ...   
41152  Airline pilots offering to stock supermarket s...             Neutral   
41153  Response to complaint not provided citing COVI...  Extremely Negative   
41154  You know itÂs getting tough when @KameronWild...            Positive   
41155  Is it wrong that the smell of hand sanitizer i...             Neutral   
41156  @TartiiCat Well new/used Rift S are going for ...            Negative   

                                       

In [17]:
#df_subset.replace({"positive":1,"negative":0},inplace=True)

In [18]:
df_subset.head(5)

,OriginalTweet,Sentiment,clean_tweets
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral,gahan
1,advice Talk to your neighbours family to excha...,Positive,advic talk neighbour famili exchang phone numb...
2,Coronavirus Australia: Woolworths to give elde...,Positive,coronaviru australia woolworth give elderli di...
3,My food stock is not the only one which is emp...,Positive,food stock onli one empti plea panic enough fo...
4,"Me, ready to go at supermarket during the #COV...",Extremely Negative,readi go supermarket dure covid outbreak becau...


# 3) Using the train_test_split function of Sklearn, Split train and test dataset - 1 Mark

In [19]:
X, y = df_subset.clean_tweets,df_subset.Sentiment
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=0)

# 4) Create pipeline and define parameters for GridSearch ( You might Refer the code below ) - 1 Mark

In [20]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])
tuned_parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': [1, 1e-1, 1e-2]
}


In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.metrics import confusion_matrix

In [22]:
text_clf.fit(X_train, y_train)
score = text_clf.score(X_test, y_test)
prediction = text_clf.predict(X_test)
print('\naccuracy: {}%'.format(round(accuracy_score(y_test, prediction)*100,2)))
print('\n',confusion_matrix(y_test, prediction))
print('\n',classification_report(y_test, prediction))
print(score)


accuracy: 38.17%

 [[  30    0  763    6  295]
 [   0   57   38    3 1233]
 [   5    2  900   20 1059]
 [   1    2  268  162 1093]
 [   1    4  277   20 1993]]

                     precision    recall  f1-score   support

Extremely Negative       0.81      0.03      0.05      1094
Extremely Positive       0.88      0.04      0.08      1331
          Negative       0.40      0.45      0.43      1986
           Neutral       0.77      0.11      0.19      1526
          Positive       0.35      0.87      0.50      2295

          accuracy                           0.38      8232
         macro avg       0.64      0.30      0.25      8232
      weighted avg       0.59      0.38      0.30      8232

0.3816812439261419


In [23]:
import joblib
joblib.dump(text_clf, 'your_pipeline.pkl')
joblib.load('your_pipeline.pkl').score(X_test, y_test)

0.3816812439261419

## 5) Perform classification (using GridSearch) - 3 Mark

In [24]:
grid_search = GridSearchCV(text_clf, tuned_parameters, cv=10, verbose=1,n_jobs=-1)
grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 36 candidates, totalling 360 fits


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'clf__alpha': [1, 0.1, 0.01],
                         'tfidf__norm': ('l1', 'l2'),
                         'tfidf__use_idf': (True, False),
                         'vect__ngram_range': [(1, 1), (1, 2), (2, 2)]},
             verbose=1)

## 6) Print the confusion matrix, accuracy, F1 score on the test dataset - 2 Mark

In [26]:
#grid_search.fit(X_train, y_train)
print('Best params are : %s' % grid_search.best_params_)
# Best training data accuracy
print('Best training accuracy: %.3f' % grid_search.best_score_)
# Predict on test data with best params
y_pred = grid_search.predict(X_test)
# Test data accuracy of model with best params
print('Test set accuracy score for best params: %.3f ' % accuracy_score(y_test, y_pred))
# Track best (highest test accuracy) model
print('\naccuracy: {}%'.format(round(accuracy_score(y_test, y_pred)*100,2)))
best_acc = accuracy_score(y_test, y_pred)
print('\n',confusion_matrix(y_test, y_pred))
print('\n',classification_report(y_test, y_pred))

Best params are : {'clf__alpha': 0.01, 'tfidf__norm': 'l2', 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}
Best training accuracy: 0.451
Test set accuracy score for best params: 0.460 

accuracy: 46.03%

 [[ 427   15  452   42  158]
 [  14  553  125   40  599]
 [ 209   72  932  166  607]
 [  41   71  282  662  470]
 [  68  314  477  221 1215]]

                     precision    recall  f1-score   support

Extremely Negative       0.56      0.39      0.46      1094
Extremely Positive       0.54      0.42      0.47      1331
          Negative       0.41      0.47      0.44      1986
           Neutral       0.59      0.43      0.50      1526
          Positive       0.40      0.53      0.45      2295

          accuracy                           0.46      8232
         macro avg       0.50      0.45      0.46      8232
      weighted avg       0.48      0.46      0.46      8232

